In [11]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j

In [ ]:
NEO4J_URI="NEO4J_URI"
NEO4J_USERNAME="NEO4J_USERNAME"
NEO4J_PASSWORD="NEO4J_PASSWORD"

In [13]:
import os
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [14]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

/tmp/ipython-input-3558848663.py:2: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph=Neo4jGraph(


In [ ]:
graph

In [ ]:
groq_api_key="groq_api_key"

In [ ]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")
llm

In [18]:
import pandas as pd

url = "https://github.com/ali-ce/datasets/raw/master/Interpol-Most-Wanted/Fugitives.csv"
df = pd.read_csv(url)

print(df.head())

                 Fugitive                   Nationality     Wanted by  \
0        Viktoryia TSUNIK                       Belarus       Belarus   
1       Adriano GIACOBONE                         Italy         Italy   
2          Sudiman SUNOTO                     Indonesia     Indonesia   
3  David Macdonald CARROL                        Canada        Canada   
4           Jason HOLLAND  United Kingdom, South Africa  South Africa   

                                          Wanted for  \
0                                       Theft, Fraud   
1  Kidnapping, Possession of firearms and/or expl...   
2              Illegal Logging, Environmental Crimes   
3              Murder, Attempted Murder, Drug Crimes   
4                                              Fraud   

                        Details of reason wanted for  \
0                            Theft by abuse of power   
1  Kidnapping, illegal detention and carrying of ...   
2                                    Illegal logging   


In [21]:
from langchain_core.documents import Document

In [22]:
def make_story_from_csv_row(row):
    name = row['Fugitive']
    nationality = row['Nationality']
    birth_year = row['Date of Birth']
    sex = row['Sex']
    details = row['Details']
    wanted_for = row['Wanted for']
    wanted_by = row['Wanted by']
    locations = row['Country believed to be in / Country of capture']
    status = row['Status']
    year = row['Year of Interpol operation']

    if sex == 'Male':
        he_she = 'He'
        him_her = 'him'
        man_woman = 'man'
    else:
        he_she = 'She'
        him_her = 'her'
        man_woman = 'woman'

    if ',' in str(nationality):
        nationality_part = f"with dual citizenship from {nationality}"
    else:
        nationality_part = f"{nationality} {man_woman}"

    story = f"""{name}, a {nationality_part} born in {birth_year}, is wanted by {wanted_by} for {wanted_for.lower()}. {details}

{he_she} has fled and is believed to be hiding in {locations}. Despite Interpol operations targeting {him_her} in {year}, {him_her.lower()} remains {status.lower()}."""
    return story

In [23]:
documents = [Document(page_content=make_story_from_csv_row(row)) for _, row in df.iterrows()]

In [24]:
all_documents = []

for _, row in df.iterrows():
    story = make_story_from_csv_row(row)
    all_documents.append(Document(page_content=story))

print(f"Created {len(all_documents)} document stories from the CSV!")
for i in range(5):
    print(f"Sample Story {i+1}:\n{all_documents[i].page_content}\n\n")

Created 95 document stories from the CSV!
Sample Story 1:
Viktoryia TSUNIK, a Belarus woman born in 1961.0, is wanted by Belarus for theft, fraud. Founder of the "Tsunik" company in Vitebsk, Belarus, between May and October 2005, Viktoryia TSUNIK, through forged false bills of lading sold 537,495 tons of stove fuel falsely labelled as diesel fuel, misappropriating about USD 130,000. In other similar cases, TSUNIK sold oil emulsion as stove fuel to three other companies, worth an estimated USD 15,000. TSUNIK was convicted and sentenced by Belarusian judicial authorities to serve a prison sentence. She speaks Russian and English.

She has fled and is believed to be hiding in Angola, Europe, Russia, Ukraine. Despite Interpol operations targeting her in 2012, her remains free.


Sample Story 2:
Adriano GIACOBONE, a Italy man born in 1957.0, is wanted by Italy for kidnapping, possession of firearms and/or explosives, receiving stolen property, wounding, theft, offences against law enforceme

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

test_documents = all_documents[:5]
short_documents = []

for doc in test_documents:
    chunks = splitter.split_text(doc.page_content)
    for chunk in chunks:
        if len(chunk.strip()) > 50:
            short_documents.append(Document(page_content=chunk))

print(f"Created {len(short_documents)} smaller document chunks")
print(f"Sample chunk: {short_documents[0].page_content[:150]}...")

Created 20 smaller document chunks
Sample chunk: Viktoryia TSUNIK, a Belarus woman born in 1961.0, is wanted by Belarus for theft, fraud. Founder of the "Tsunik" company in Vitebsk, Belarus, between ...


In [26]:
!pip install --upgrade --quiet langchain_experimental

In [27]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Person", "Location", "Organization", "Crime"],
    allowed_relationships=["WANTED_FOR", "HIDING_IN", "CITIZEN_OF", "WANTED_BY"],
    strict_mode=False
)

print("Transformer configured with specific node and relationship types")

Transformer configured with specific node and relationship types


In [29]:
print("Testing with one document...")
try:
    test_graph = llm_transformer.convert_to_graph_documents([short_documents[0]])
    print("SUCCESS! Single document worked")
    print(f"Found {len(test_graph[0].nodes)} nodes and {len(test_graph[0].relationships)} relationships")

    print("Now trying with 3 documents...")
    graph_documents = llm_transformer.convert_to_graph_documents(short_documents[:3])
    print(f"SUCCESS! Processed {len(graph_documents)} documents")

except Exception as e:
    print(f"Error: {e}")
    print("Let's try a different approach...")

Testing with one document...
SUCCESS! Single document worked
Found 3 nodes and 4 relationships
Now trying with 3 documents...
SUCCESS! Processed 3 documents


In [30]:
graph_documents[0].relationships

[Relationship(source=Node(id='Viktoryia Tsunik', type='Person', properties={}), target=Node(id='Belarus', type='Location', properties={}), type='CITIZEN_OF', properties={}),
 Relationship(source=Node(id='Viktoryia Tsunik', type='Person', properties={}), target=Node(id='Belarus', type='Location', properties={}), type='WANTED_BY', properties={}),
 Relationship(source=Node(id='Viktoryia Tsunik', type='Person', properties={}), target=Node(id='Tsunik', type='Organization', properties={}), type='FOUNDER_OF', properties={}),
 Relationship(source=Node(id='Viktoryia Tsunik', type='Person', properties={}), target=Node(id='Vitebsk', type='Location', properties={}), type='CITIZEN_OF', properties={}),
 Relationship(source=Node(id='Viktoryia Tsunik', type='Person', properties={}), target=Node(id='Tsunik', type='Organization', properties={}), type='FOUNDER_OF', properties={}),
 Relationship(source=Node(id='Viktoryia Tsunik', type='Person', properties={}), target=Node(id='Belarus', type='Location', pr

In [31]:
graph

In [32]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Fugitive {fugitiveId: STRING, name: STRING, dateOfBirth: DATE, currentAge: INTEGER, sex: STRING, status: STRING, details: STRING, yearOfInterpolOperation: STRING, source: STRING, interpolRedNoticeProfile: STRING, image: STRING, reasonDetails: STRING}
Country {name: STRING}
Organization {name: STRING}
Crime {name: STRING}
Relationship properties:

The relationships:
(:Fugitive)-[:HAS_NATIONALITY]->(:Country)
(:Fugitive)-[:WANTED_BY]->(:Organization)
(:Fugitive)-[:WANTED_FOR]->(:Crime)
(:Fugitive)-[:BELIEVED_IN_COUNTRY]->(:Country)


In [33]:
node_types = graph.query("MATCH (n) RETURN DISTINCT labels(n) as node_types")
print("Node types in the graph:")
for record in node_types:
    print(f"  - {record['node_types']}")

Node types in the graph:
  - ['Fugitive']
  - ['Country']
  - ['Organization']
  - ['Crime']


In [34]:
counts = graph.query("MATCH (n) RETURN labels(n) as type, count(n) as count")
print("\nNode counts:")
for record in counts:
    print(f"  - {record['type']}: {record['count']}")


Node counts:
  - ['Fugitive']: 95
  - ['Country']: 85
  - ['Organization']: 50
  - ['Crime']: 49


In [38]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    allow_dangerous_requests=True,  # Enables LLM to send tool calls directly (e.g., extract nodes/relationships), even if the format isn't perfectly validated. Useful for prototyping.
    return_intermediate_steps=True
)

In [39]:
test_questions = [
    "How many fugitives are in the database?",
    "Show me fugitives from the United States",
    "What is the status of most fugitives?",
    "How many male vs female fugitives are there?",
    "Show me fugitives from Belarus"
]

In [40]:
for i, question in enumerate(test_questions, 1):
    print(f"\n Test {i} ")
    print(f"Question: {question}")

    try:
        response = chain.invoke({"query": question})
        print(f"Generated Query: {response['intermediate_steps'][0]['query']}")
        print(f"Database Result: {response['intermediate_steps'][1]['context']}")
        print(f"Final Answer: {response['result']}")

    except Exception as e:
        print(f"Error: {e}")

    print("\n \n")


 Test 1 
Question: How many fugitives are in the database?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (f: Fugitive) RETURN COUNT(f)
Full Context:
[{'COUNT(f)': 95}]

> Finished chain.
Generated Query: MATCH (f: Fugitive) RETURN COUNT(f)
Database Result: [{'COUNT(f)': 95}]
Final Answer: I don't know the answer.

 


 Test 2 
Question: Show me fugitives from the United States


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (f: Fugitive)-[:HAS_NATIONALITY]->(c: Country) WHERE c.name = "United States" RETURN f
Full Context:
[{'f': {'currentAge': 46, 'image': 'http://media.trb.com/media/photo/2011-06/62716675.jpg', 'fugitiveId': 'Kenneth Andrew CRAIG', 'sex': 'Male', 'name': 'Kenneth Andrew CRAIG', 'dateOfBirth': neo4j.time.Date(1968, 1, 1), 'details': 'CRAIG is wanted for the alleged indecent assault of two teenage boys in October 1998 in Florida, United States. The victims were both under 16 at the time. It is also alleged that CRAIG forced 

In [47]:
def chat_with_fugitives():
    print("🔍 GraphJusticeQ&A")
    print("Type 'quit' to exit\n")

    while True:
        question = input("You: ")

        if question.lower() == 'quit':
            print("Goodbye!")
            break

        if question.strip() == "":
            continue

        if question.lower() in ['hi', 'hello', 'hey']:
            print("GraphJusticeQ&A: Hi! Ask me about fugitives.")
            continue

        try:
            print("🤖 Searching...")
            response = chain.invoke({"query": question})
            print(f"GraphJusticeQ&A: {response['result']}")
            if response.get('intermediate_steps'):
                query = response['intermediate_steps'][0]['query']
                simple_explanation = explain_query(query)
                print(f"(Query: {simple_explanation})")

        except Exception as e:
            print(f"Error: {e}")

        print("-" * 30)

def explain_query(query):
    query = query.lower()

    if "count" in query:
        return "Counted fugitives"
    elif "wanted_by" in query:
        return "Found most wanted"
    elif "deceased" in query:
        return "Searched deceased status"
    elif "name" in query:
        return "Looked up by name"
    elif "nationality" in query:
        return "Searched nationality"
    elif "crime" in query:
        return "Searched crimes"
    else:
        return "Searched database"

chat_with_fugitives()

🔍 GraphJusticeQ&A
Type 'quit' to exit

You: hi
GraphJusticeQ&A: Hi! Ask me about fugitives.
You: Tell me everything about him Steven Douglas SKINNER 
🤖 Searching...
Error: {code: Neo.ClientError.Statement.SyntaxError} {message: Variable `Country` not defined (line 1, column 130 (offset: 129))
"MATCH (f: Fugitive {name: "Steven Douglas SKINNER"})-[]->(:Country), (f)-[]->(:Organization), (f)-[]->(:Crime) RETURN f, COLLECT(Country.name) as nationality, COLLECT(Organization.name) as wantedBy, COLLECT(Crime.name) as wantedFor"
                                                                                                                                  ^}
------------------------------
You: Show me fugitives from the United States
🤖 Searching...
GraphJusticeQ&A: Kenneth Andrew CRAIG, Bailey Martin COULTER, Christopher Ward DEININGER, John Edward HAMILTON, Frank Cornelis LEFRANDT Jr., Catherine Elizabeth GREIG, Robert William FISHER are fugitives from the United States.
(Query: Counted fug